In [ ]:
subscription_id = "xxx-xxx-xxx-xxx-xxx" 
resource_group = "Machine_Learning_Experiment" 
workspace_name = "Experiment_Workspace" 
workspace_region = "westeurope" 

In [ ]:
import azureml.core
print('azureml.core.VERSION: ', azureml.core.VERSION)

from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region, 
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

In [ ]:
!cat .azureml/config.json

In [ ]:
%%writefile summarizer_service.py

import re
import nltk
import unicodedata
from gensim.summarization import summarize, keywords

def clean_and_parse_document(document):
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError("Document is not string or unicode.")
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

def summarize_text(text, summary_ratio = None, word_count = 30):
    sentences = clean_and_parse_document(text)
    cleaned_text = ' '.join(sentences)
    summary = summarize(cleaned_text, split = True, ratio = summary_ratio, word_count = word_count)
    return summary 

def init():  
    nltk.download('all')
    return

def run(input_str):
    try:
        return summarize_text(input_str)
    except Exception as e:
        return (str(e))

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(pip_packages = ['gensim','nltk'])

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Summarization'}, 
    description = 'Summarizes text.')

In [ ]:
service_name = "summarizer"
runtime = "python"
driver_file = "summarizer_service.py"
conda_file = "mydeployenv.yml"

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

In [ ]:
webservice = Webservice.deploy(
  workspace = ws, 
  name = service_name, 
  model_paths = [],
  deployment_config = aci_config,
  image_config = image_config, 
  )

webservice.wait_for_deployment(show_output = True)

In [ ]:
example = """
I would like to thank you for a wonderful stay at the Hotel California.   
The room we stayed in was very nice and had plenty of room for the whole family and the beds were especially comfortable.  
The kids loved going to the Kids Club and the swimming pools all day which gave us some time to relax and have a few cocktails before eating at the Bistro restaurant. The food was great and the kids menu was priced very reasonably.
A big thank you to all the staff who couldn't do enough for us and were polite and friendly throughout our stay. 
We had a great family holiday and can't wait to book again for next year. 
"""

In [ ]:
result = webservice.run(input_data = example)
print(result)

In [ ]:
webservice.scoring_uri